<a href="https://colab.research.google.com/github/FelixLeChat92/MermaidGen-Colab-Jupyter/blob/main/MermaidGen_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mermaid Diagram Generation

This notebook allows you to generate Mermaid diagrams in different formats (SVG, PNG) and export them. It uses the online services [mermaid.live](https://mermaid.live/), [kroki.io](https://kroki.io/), and [mermaid.ink](https://mermaid.ink/) for rendering the diagrams.

## Table of Contents

- [Importing Libraries](#importation)
- [Defining the Mermaid Diagram](#diagramme-mermaid)
- [Generating an Edit Link on mermaid.live](#lien-mermaid-live)
- [Exporting as SVG with kroki.io](#export-svg-kroki)
- [Exporting with mermaid.ink](#export-mermaid-ink)
  - [SVG Format](#export-svg-mermaid-ink)
  - [PNG Format](#export-png-mermaid-ink)
- [Conclusion](#conclusion)

## Importing Libraries <a name="importation"></a>

The following libraries are used in this notebook:

- `base64`: for encoding and decoding in base64
- `zlib`: for compressing and decompressing data
- `json`: for manipulating JSON data
- `requests`: for making HTTP requests
- `IPython.display`: for displaying images in the notebook

In [1]:
import base64
import sys
import json
import zlib
from IPython.display import SVG, Image
import requests

## Defining the Mermaid Diagram <a name="diagramme-mermaid"></a>

The Mermaid code for the diagram to be generated is defined in the `mermaid_code` variable.
Modify this variable to change the diagram. You can use the different types of Mermaid diagrams available and special characters supported by UTF-8.

```mermaid
mindmap
  root((Root))
    Branch 1
      Node 1
      Node 2
    Branch 2
      Node 3
      Node 4
```

In [2]:
# Usage example
mermaid_code = '''
mindmap
  root((Hypertension Medications))
    Diuretics
      Thiazides
        Hydrochlorothiazide
        Indapamide
      Loop Diuretics
'''

## Generating an Edit Link on mermaid.live <a name="lien-mermaid-live"></a>

The `genPakoLink` function generates a link to the online editor [mermaid.live](https://mermaid.live/) pre-filled with the previously defined Mermaid code.

Click on the generated link to open the diagram in mermaid.live and edit it interactively.

In [ ]:
def js_btoa(data):
    return base64.b64encode(data)

def pako_deflate(data):
    compress = zlib.compressobj(9, zlib.DEFLATED, 15, 8, zlib.Z_DEFAULT_STRATEGY)
    compressed_data = compress.compress(data)
    compressed_data += compress.flush()
    return compressed_data

def genPakoLink(graphMarkdown: str):
    jGraph = {"code": graphMarkdown, "mermaid": {"theme": "standard"}}
    byteStr = json.dumps(jGraph).encode('utf-8')
    deflated = pako_deflate(byteStr)
    dEncode = js_btoa(deflated)
    link_code = dEncode.decode('ascii')
    return link_code

mermaid_link = genPakoLink(mermaid_code)
print("mermaid.live link:")
print('http://mermaid.live/edit#pako:' + mermaid_link)

## Exporting as SVG with kroki.io <a name="export-svg-kroki"></a>

The `encode_mermaid_to_kroki_url` function generates a link to an SVG image of the diagram using the [kroki.io](https://kroki.io/) service.

The following code downloads this SVG image and saves it in the current directory.

In [ ]:
def encode_mermaid_to_kroki_url(mermaid_diagram, compression_level=9):
    # Compress the Mermaid diagram with zlib
    mermaid_compressed = zlib.compress(mermaid_diagram.encode('utf-8'), compression_level)

    # Encode the compressed diagram in URL-safe base64
    mermaid_base64 = base64.urlsafe_b64encode(mermaid_compressed).decode('utf-8')

    # Construct the complete Kroki URL
    kroki_url = f"https://kroki.io/mermaid/svg/{mermaid_base64}"

    return kroki_url

kroki_link = encode_mermaid_to_kroki_url(mermaid_code)
print(kroki_link)

string = kroki_link[-8:]
filename = string + ".svg"

response = requests.get(kroki_link)
if response.status_code == 200:
  # Save the SVG image to a file
  with open(filename, "wb") as file:
    file.write(response.content)
  print(f"The SVG image has been saved as '{filename}'")
else:
  print("Error downloading the SVG image")
SVG(response.text)

## Export with mermaid.ink <a name="export-mermaid-ink"></a>

### SVG Format <a name="export-svg-mermaid-ink"></a>

This section generates an SVG image of the diagram using [mermaid.ink](https://mermaid.ink/), displays it in the notebook, and saves it to a file.

In [ ]:
def mermaid_ink_string(graph):
  graphbytes = graph.encode("utf-8")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("utf-8")
  string = base64_string
  return string

string = mermaid_ink_string(mermaid_code)
url_image_svg = "https://mermaid.ink/svg/" + string
print("Link to the Mermaid image: " + url_image_svg)

filename = string[:8] + ".svg"

response = requests.get(url_image_svg)
if response.status_code == 200:
  # Save the SVG image to a file
  with open(filename, "wb") as file:
    file.write(response.content)
  print(f"The SVG image has been saved as '{filename}'")
else:
  print("Error downloading the SVG image")
SVG(response.text)

## PNG - Low Quality
### PNG Format <a name="export-png-mermaid-ink"></a>

Similar to the previous section but generates a PNG image (lower quality).

In [ ]:
def mermaid_ink_string(graph):
  graphbytes = graph.encode("utf-8")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("utf-8")
  string = base64_string
  return string

string = mermaid_ink_string(mermaid_code)

url_image_png = "https://mermaid.ink/img/" + string
print("Link to the Mermaid image: " + url_image_png)

filename = string[:8] + ".png"

response = requests.get(url_image_png)

display(Image(url=url_image_png))

response = requests.get(url_image_png)
if response.status_code == 200:
  # Save the PNG image to a file
  with open(filename, "wb") as file:
    file.write(response.content)
  print(f"The PNG image has been saved as '{filename}'")
else:
  print("Error downloading the PNG image")

## Conclusion <a name="Conclusion"></a>

This notebook facilitates the generation and export of Mermaid diagrams in different formats. You can adapt it to your needs by modifying the Mermaid code in the `mermaid_code` variable.

## Resources

- Mermaid Documentation: [https://mermaid.js.org/](https://mermaid.js.org/)
- Online editor mermaid.live: [https://mermaid.live/](https://mermaid.live/)
- Image rendering service kroki.io: [https://kroki.io/](https://kroki.io/)
- mermaid.ink service: [https://mermaid.ink/](https://mermaid.ink/)